# PRACTICA CALIFICADA N°3

Leonarddo Carlos López Solis
20232345

# CLEANING AND FORMATING

In [64]:
import pandas as pd
linkEdu="https://github.com/LeonarddoCLS/PC3_entrega/raw/refs/heads/main/Education%20expenditures%20(1).csv"
linkElec="https://github.com/LeonarddoCLS/PC3_entrega/raw/refs/heads/main/Electricity%20-%20installed%20generating%20capacity.csv"
linkEnergy="https://github.com/LeonarddoCLS/PC3_entrega/raw/refs/heads/main/Energy%20consumption%20per%20capita.csv"
linkReserve="https://github.com/LeonarddoCLS/PC3_entrega/raw/refs/heads/main/Reserves%20of%20foreign%20exchange%20and%20gold.csv"

education=pd.read_csv(linkEdu)
electricity=pd.read_csv(linkElec)
energy=pd.read_csv(linkEnergy)
reserve=pd.read_csv(linkReserve)




En el caso existan valores perdidos, los eliminamos con este comando

In [65]:
education_1 = education.dropna()
electricity_1 = electricity.dropna()
energy_1 = energy.dropna()
reserve_1 = reserve.dropna()

Detectamos que dos columns tienen leading space

In [66]:
electricity_1.columns = electricity_1.columns.str.strip()
energy_1.columns = energy_1.columns.str.strip()

En caso hubieran duplicados en los países, los eliminamos de la siguiente forma comando

In [67]:
delete_Duplicates = lambda df: df.drop_duplicates(subset=['name'])
delete_Duplicates(education_1)
delete_Duplicates(electricity_1)
delete_Duplicates(energy_1)
delete_Duplicates(reserve_1)


,name,slug,value,date_of_information,ranking,region
0,China,china,"$3,450,000,000,000",2023,1,East and Southeast Asia
1,Japan,japan,"$1,295,000,000,000",2023,2,East and Southeast Asia
2,Switzerland,switzerland,"$863,892,000,000",2023,3,Europe
3,United States,united-states,"$773,426,000,000",2023,4,North America
4,India,india,"$627,793,000,000",2023,5,South Asia
...,...,...,...,...,...,...
190,Anguilla,anguilla,"$76,380,000",2017,191,Central America and the Caribbean
191,Burkina Faso,burkina-faso,"$49,000,000",2017,192,Africa
192,Montserrat,montserrat,"$47,580,000",2017,193,Central America and the Caribbean
193,Sao Tome and Principe,sao-tome-and-principe,"$46,247,000",2023,194,Africa



---Primero eliminamos leading, tailing y espacios internos en blanco



In [68]:
import re
clean_column_names = lambda df: df.columns.astype(str).str.strip().str.replace(pat=r'\s', repl='_', regex=True)

education_1.columns = clean_column_names(education_1)
electricity_1.columns = clean_column_names(electricity_1)
energy_1.columns = clean_column_names(energy_1)
reserve_1.columns = clean_column_names(reserve_1)

--- Limpiamos las columnas de variables tipo texto. Para ello, primero guardo el nombre de las "Column names"

In [69]:
common_columns = set(education.columns.intersection(electricity.columns).intersection(energy.columns).intersection(reserve.columns))
common_text_columns=common_columns - {"date_of_information", "ranking"}
list_common_text=list(common_text_columns)
list_common_text

['region', 'name', 'slug']

In [70]:
!pip install unidecode

Eliminamos caracteres non_ASCII utilizando unicode

In [71]:
from unidecode import unidecode
apply_unidecode = lambda col:col.apply(unidecode).str.replace(pat='-', repl='_', regex=True)
education_1[list_common_text]=education_1.loc[:, list_common_text].apply(apply_unidecode)
electricity_1[list_common_text]=electricity_1.loc[:, list_common_text].apply(apply_unidecode)
energy_1[list_common_text]=energy_1.loc[:, list_common_text].apply(apply_unidecode)
reserve_1[list_common_text]=reserve_1.loc[:, list_common_text].apply(apply_unidecode)

first_group=["name", "region"]
turn_mayus=lambda col:col.str.upper()
education_1[first_group]=education_1.loc[:,first_group ].apply(turn_mayus)
electricity_1[first_group]=electricity_1.loc[:,first_group ].apply(turn_mayus)
energy_1[first_group]=energy_1.loc[:,first_group ].apply(turn_mayus)
reserve_1[first_group]=reserve_1.loc[:, first_group].apply(turn_mayus)






Limpiamos columnas numéricas


In [72]:
get_numeric_colums= lambda df:list(set(df.columns)-set(common_text_columns))
education_1_numeric=get_numeric_colums(education_1)
electricity_1_numeric=get_numeric_colums(electricity_1)
energy_1_numeric=get_numeric_colums(energy_1)
reserve_1_numeric=get_numeric_colums(reserve_1)


In [73]:
apply_pd_numeric=lambda col:col.astype(str).str.replace(pat=',|\$', repl='', regex=True).apply(pd.to_numeric)


education_1[education_1_numeric]=education_1.loc[:,education_1_numeric].apply(apply_pd_numeric)
electricity_1[electricity_1_numeric]=electricity_1.loc[:,electricity_1_numeric].apply(apply_pd_numeric)
energy_1[energy_1_numeric]=energy_1.loc[:,energy_1_numeric].apply(apply_pd_numeric)
reserve_1[reserve_1_numeric]=reserve_1.loc[:,reserve_1_numeric].apply(apply_pd_numeric)




# APPLYING FUZZY MERGING APPROACH:

Se eligió esta estrategia debido a que es la que permite recuperar un número mayor de casos al aplicar el merging.

1) Eliminamos la columna slug y date_of_information, con el fin de quedarnos solo con las variables principales

In [74]:
delete_date = lambda df: df.drop(columns=['date_of_information']).drop(columns=['ranking']).drop(columns=['slug'])
education_1=delete_date(education_1)
electricity_1=delete_date(electricity_1)
energy_1=delete_date(energy_1)
reserve_1=delete_date(reserve_1)

2) Mejorando los nombres de ciertas columnas para diferenciarlas o esclarecer el significado del valor

In [75]:
education_1 = education_1.rename(columns={'%_of_GDP':'education_expenditure(%GDP)'})
electricity_1 = electricity_1.rename(columns={'kW':'power_generation(kW)'})
energy_1 = energy_1.rename(columns={'Btu/person':'Btu/person_consumption($)'})
reserve_1 = reserve_1.rename(columns={'value':'financial_assets_value($)'})


3) Empezamos a hacer merge a education_1 y electricity_1

3.1) Visualizamos que países se encuentran en únicamente en el primer o el segundo dataframe

In [76]:
only_education=set(education_1.name)-set(electricity_1.name)
only_education

{'ANDORRA',
 'ANGUILLA',
 'CURACAO',
 'LIECHTENSTEIN',
 'MARSHALL ISLANDS',
 'MICRONESIA, FEDERATED STATES OF',
 'MONACO',
 'PALAU',
 'SAN MARINO'}

In [77]:
only_electricity=set(electricity_1.name)-set(education_1.name)
only_electricity

{'AMERICAN SAMOA',
 'BOSNIA AND HERZEGOVINA',
 'CUBA',
 'EQUATORIAL GUINEA',
 'ERITREA',
 'FALKLAND ISLANDS (ISLAS MALVINAS)',
 'FRENCH POLYNESIA',
 'GIBRALTAR',
 'GUAM',
 'KOREA, NORTH',
 'KOSOVO',
 'LIBYA',
 'MONTENEGRO',
 'NEW CALEDONIA',
 'NIUE',
 'NORTH MACEDONIA',
 'SAINT HELENA, ASCENSION, AND TRISTAN DA CUNHA',
 'SAINT PIERRE AND MIQUELON',
 'SUDAN',
 'SYRIA',
 'TAIWAN',
 'VIRGIN ISLANDS',
 'YEMEN'}

3.2) Aplicamos el enfoque thefuzz para ver posibles pares de países que sean compatibles

In [78]:
!pip install thefuzz

In [79]:
from thefuzz import process as fz
[(c,fz.extractOne(c,only_education )) for c in sorted(only_electricity)]

[('AMERICAN SAMOA', ('MONACO', 60)),
 ('BOSNIA AND HERZEGOVINA', ('ANDORRA', 53)),
 ('CUBA', ('CURACAO', 68)),
 ('EQUATORIAL GUINEA', ('PALAU', 54)),
 ('ERITREA', ('MICRONESIA, FEDERATED STATES OF', 51)),
 ('FALKLAND ISLANDS (ISLAS MALVINAS)', ('MARSHALL ISLANDS', 86)),
 ('FRENCH POLYNESIA', ('MICRONESIA, FEDERATED STATES OF', 48)),
 ('GIBRALTAR', ('PALAU', 54)),
 ('GUAM', ('ANGUILLA', 50)),
 ('KOREA, NORTH', ('ANDORRA', 51)),
 ('KOSOVO', ('MICRONESIA, FEDERATED STATES OF', 36)),
 ('LIBYA', ('ANGUILLA', 51)),
 ('MONTENEGRO', ('MONACO', 60)),
 ('NEW CALEDONIA', ('MONACO', 54)),
 ('NIUE', ('LIECHTENSTEIN', 51)),
 ('NORTH MACEDONIA', ('MONACO', 60)),
 ('SAINT HELENA, ASCENSION, AND TRISTAN DA CUNHA', ('ANDORRA', 57)),
 ('SAINT PIERRE AND MIQUELON', ('SAN MARINO', 51)),
 ('SUDAN', ('SAN MARINO', 68)),
 ('SYRIA', ('MICRONESIA, FEDERATED STATES OF', 54)),
 ('TAIWAN', ('LIECHTENSTEIN', 54)),
 ('VIRGIN ISLANDS', ('MARSHALL ISLANDS', 63)),
 ('YEMEN', ('LIECHTENSTEIN', 45))]

3.3)Dado que el máximo porcentaje de similitud en realidad no corresponde a nombres compatibles y que no se visualiza ningún otro caso compatible, se verificará en la otra dirección sin recuperar ninguno

In [80]:
[(c,fz.extractOne(c,only_electricity )) for c in sorted(only_education)]

[('ANDORRA', ('SAINT HELENA, ASCENSION, AND TRISTAN DA CUNHA', 57)),
 ('ANGUILLA', ('FALKLAND ISLANDS (ISLAS MALVINAS)', 56)),
 ('CURACAO', ('CUBA', 68)),
 ('LIECHTENSTEIN', ('TAIWAN', 54)),
 ('MARSHALL ISLANDS', ('FALKLAND ISLANDS (ISLAS MALVINAS)', 86)),
 ('MICRONESIA, FEDERATED STATES OF', ('SYRIA', 54)),
 ('MONACO', ('AMERICAN SAMOA', 60)),
 ('PALAU', ('EQUATORIAL GUINEA', 54)),
 ('SAN MARINO', ('SUDAN', 68))]

En ninguno de los dos casos hay paises compatibles

3.4)Eliminaremos todos esas filas, debido a que no se tendría información completa para ninguno de esos casos


In [81]:
only_education = list(only_education)
only_electricity = list(only_electricity)
education_2 = education_1[~education_1['name'].isin(only_education)]
electricity_2 = electricity_1[~electricity_1['name'].isin(only_electricity)]

3.5) Dado que ahora ambos tienen las mismas filas, aplicaremos merging left join. Se puede elegir, cualquiera de los otros 3 tipos: inner join, right join, outer join.

In [82]:
first_merge=education_2.merge(electricity_2, how='left', left_on='name', right_on='name')
first_merge=first_merge.drop(columns=['region_y'])
first_merge

,name,education_expenditure(%GDP),region_x,power_generation(kW)
0,SOLOMON ISLANDS,12.8,AUSTRALIA AND OCEANIA,37000
1,KIRIBATI,12.4,AUSTRALIA AND OCEANIA,12000
2,GREENLAND,10.2,NORTH AMERICA,190000
3,BOLIVIA,9.8,SOUTH AMERICA,4375000
4,NAMIBIA,9.6,AFRICA,646000
...,...,...,...,...
183,HAITI,1.4,CENTRAL AMERICA AND THE CARIBBEAN,472000
184,PAPUA NEW GUINEA,1.4,EAST AND SOUTHEAST ASIA,1148000
185,VENEZUELA,1.3,SOUTH AMERICA,33493000
186,NIGERIA,0.5,AFRICA,4094000


4) Continuamos con merge entre energy y reserve


4.1) Visualizamos que países se encuentran en únicamente en el primer o el segundo dataframe


In [83]:
only_energy=set(energy_1.name)-set(reserve_1.name)
only_energy

{'AMERICAN SAMOA',
 'BERMUDA',
 'BRITISH VIRGIN ISLANDS',
 'GUAM',
 'KIRIBATI',
 'KOREA, NORTH',
 'PUERTO RICO'}

In [84]:
only_reserve=set(reserve_1.name)-set(energy_1.name)
only_reserve

{'ANGUILLA',
 'ICELAND',
 'MICRONESIA, FEDERATED STATES OF',
 'MONTSERRAT',
 'PALAU',
 'SAN MARINO',
 'TRINIDAD AND TOBAGO'}

4.2) Aplicamos el enfoque thefuzz para ver posibles pares de países que sean compatibles





In [85]:
[(c,fz.extractOne(c,only_energy )) for c in sorted(only_reserve)]

[('ANGUILLA', ('GUAM', 50)),
 ('ICELAND', ('BRITISH VIRGIN ISLANDS', 64)),
 ('MICRONESIA, FEDERATED STATES OF', ('PUERTO RICO', 47)),
 ('MONTSERRAT', ('KOREA, NORTH', 36)),
 ('PALAU', ('PUERTO RICO', 51)),
 ('SAN MARINO', ('AMERICAN SAMOA', 55)),
 ('TRINIDAD AND TOBAGO', ('KIRIBATI', 43))]

4.3) Dado que el máximo porcentaje de similitud en realidad no corresponde a nombres compatibles y que no se visualiza ningún otro caso compatible, se verificará en la otra dirección sin recuperar ninguno

In [86]:
[(c,fz.extractOne(c,only_reserve )) for c in sorted(only_energy)]

[('AMERICAN SAMOA', ('SAN MARINO', 55)),
 ('BERMUDA', ('MICRONESIA, FEDERATED STATES OF', 39)),
 ('BRITISH VIRGIN ISLANDS', ('ICELAND', 64)),
 ('GUAM', ('ANGUILLA', 50)),
 ('KIRIBATI', ('TRINIDAD AND TOBAGO', 43)),
 ('KOREA, NORTH', ('MICRONESIA, FEDERATED STATES OF', 39)),
 ('PUERTO RICO', ('PALAU', 51))]

4.4) En ambos casos no existe un par compatible, por lo que se procesderá a borrar todas esas filas de only_energy y only_reserve

In [87]:
only_energy = list(only_energy)
only_reserve = list(only_reserve)
energy_2 = energy_1[~energy_1['name'].isin(only_energy)]
reserve_2 = reserve_1[~reserve_1['name'].isin(only_reserve)]


In [88]:
second_merge=energy_2.merge(reserve_2, how='left', left_on='name', right_on='name')
second_merge=second_merge.drop(columns=['region_y'])
second_merge

,name,Btu/person_consumption($),region_x,financial_assets_value($)
0,QATAR,814308000,MIDDLE EAST,51539000000
1,SINGAPORE,643259000,EAST AND SOUTHEAST ASIA,359835000000
2,BAHRAIN,554202000,MIDDLE EAST,5118000000
3,UNITED ARAB EMIRATES,450432000,MIDDLE EAST,189491000000
4,BRUNEI,403365000,EAST AND SOUTHEAST ASIA,4483000000
...,...,...,...,...
183,CHAD,1502000,AFRICA,211591000
184,"CONGO, DEMOCRATIC REPUBLIC OF THE",1305000,AFRICA,5104000000
185,CENTRAL AFRICAN REPUBLIC,954000,AFRICA,374405000
186,BURUNDI,946000,AFRICA,90350000


5) Finalmente, aplicamos merge para second_merge y first_merge

5.1) Visualizamos que países se encuentran en únicamente en el primer o el segundo dataframe


In [89]:
only_first=set(first_merge.name)-set(second_merge.name)
only_first

{'BERMUDA',
 'BRITISH VIRGIN ISLANDS',
 'COOK ISLANDS',
 'FAROE ISLANDS',
 'GREENLAND',
 'ICELAND',
 'KIRIBATI',
 'MONTSERRAT',
 'NAURU',
 'PUERTO RICO',
 'TRINIDAD AND TOBAGO',
 'TURKS AND CAICOS ISLANDS'}

In [90]:
only_second=set(second_merge.name)-set(first_merge.name)
only_second

{'BOSNIA AND HERZEGOVINA',
 'CUBA',
 'EQUATORIAL GUINEA',
 'ERITREA',
 'KOSOVO',
 'LIBYA',
 'MONTENEGRO',
 'NORTH MACEDONIA',
 'SUDAN',
 'SYRIA',
 'TAIWAN',
 'YEMEN'}

5.2) Aplicamos el enfoque thefuzz para ver posibles pares de países que sean compatibles



In [91]:
[(c,fz.extractOne(c,only_first )) for c in sorted(only_second)]

[('BOSNIA AND HERZEGOVINA', ('ICELAND', 51)),
 ('CUBA', ('BERMUDA', 51)),
 ('EQUATORIAL GUINEA', ('BERMUDA', 49)),
 ('ERITREA', ('PUERTO RICO', 51)),
 ('KOSOVO', ('TURKS AND CAICOS ISLANDS', 43)),
 ('LIBYA', ('KIRIBATI', 54)),
 ('MONTENEGRO', ('MONTSERRAT', 60)),
 ('NORTH MACEDONIA', ('NAURU', 45)),
 ('SUDAN', ('TRINIDAD AND TOBAGO', 54)),
 ('SYRIA', ('MONTSERRAT', 54)),
 ('TAIWAN', ('TRINIDAD AND TOBAGO', 54)),
 ('YEMEN', ('GREENLAND', 54))]

5.3) Dado que el máximo porcentaje de similitud en realidad no corresponde a nombres compatibles y que no se visualiza ningún otro caso compatible, se verificará en la otra dirección sin recuperar ninguno

In [92]:
[(c,fz.extractOne(c,only_second )) for c in sorted(only_first)]

[('BERMUDA', ('CUBA', 51)),
 ('BRITISH VIRGIN ISLANDS', ('SUDAN', 54)),
 ('COOK ISLANDS', ('SUDAN', 54)),
 ('FAROE ISLANDS', ('SUDAN', 54)),
 ('GREENLAND', ('YEMEN', 54)),
 ('ICELAND', ('BOSNIA AND HERZEGOVINA', 51)),
 ('KIRIBATI', ('SYRIA', 54)),
 ('MONTSERRAT', ('MONTENEGRO', 60)),
 ('NAURU', ('BOSNIA AND HERZEGOVINA', 51)),
 ('PUERTO RICO', ('ERITREA', 51)),
 ('TRINIDAD AND TOBAGO', ('TAIWAN', 54)),
 ('TURKS AND CAICOS ISLANDS', ('SUDAN', 54))]

5.4) En ambos casos no existe un par compatible, por lo que se procesderá a borrar todas esas filas de first and second merge

In [93]:
only_first = list(only_first)
only_second = list(only_second)
first_merge = first_merge[~first_merge['name'].isin(only_first)]
# Filter the second_merge DataFrame using the boolean Series
second_merge = second_merge[~second_merge['name'].isin(only_second)]

In [94]:
final_merge=first_merge.merge(second_merge, how='left', left_on='name', right_on='name')
final_merge=final_merge.drop(columns=['region_x_y'])
final_merge

,name,education_expenditure(%GDP),region_x_x,power_generation(kW),Btu/person_consumption($),financial_assets_value($)
0,SOLOMON ISLANDS,12.8,AUSTRALIA AND OCEANIA,37000,5655000,688220000
1,BOLIVIA,9.8,SOUTH AMERICA,4375000,29340000,1800000000
2,NAMIBIA,9.6,AFRICA,646000,21734000,2956000000
3,SIERRA LEONE,9.1,AFRICA,149000,2301000,495699000
4,BELIZE,8.7,CENTRAL AMERICA AND THE CARIBBEAN,220000,30752000,473729000
...,...,...,...,...,...,...
171,HAITI,1.4,CENTRAL AMERICA AND THE CARIBBEAN,472000,3486000,2586000000
172,PAPUA NEW GUINEA,1.4,EAST AND SOUTHEAST ASIA,1148000,8781000,3901000000
173,VENEZUELA,1.3,SOUTH AMERICA,33493000,54474000,9794000000
174,NIGERIA,0.5,AFRICA,4094000,7993000,35564000000


6) Finalmente hacemos unos últimos arreglos a los nombres de las columnas

In [95]:
final_merge= final_merge.rename(columns={'name': 'Country'})
final_merge= final_merge.rename(columns={'region_x_x': 'Region'})

In [96]:
region = final_merge['Region']
final_merge=final_merge.drop(columns=['Region'])
final_merge.insert(1, 'Region', region)


Guardamos una copia para no regresar hasta el inicio

In [97]:
my_dataframe=final_merge.copy()
my_dataframe

,Country,Region,education_expenditure(%GDP),power_generation(kW),Btu/person_consumption($),financial_assets_value($)
0,SOLOMON ISLANDS,AUSTRALIA AND OCEANIA,12.8,37000,5655000,688220000
1,BOLIVIA,SOUTH AMERICA,9.8,4375000,29340000,1800000000
2,NAMIBIA,AFRICA,9.6,646000,21734000,2956000000
3,SIERRA LEONE,AFRICA,9.1,149000,2301000,495699000
4,BELIZE,CENTRAL AMERICA AND THE CARIBBEAN,8.7,220000,30752000,473729000
...,...,...,...,...,...,...
171,HAITI,CENTRAL AMERICA AND THE CARIBBEAN,1.4,472000,3486000,2586000000
172,PAPUA NEW GUINEA,EAST AND SOUTHEAST ASIA,1.4,1148000,8781000,3901000000
173,VENEZUELA,SOUTH AMERICA,1.3,33493000,54474000,9794000000
174,NIGERIA,AFRICA,0.5,4094000,7993000,35564000000


# REPORT AVERAGE, MIN AND MAX VALUES

Primera variable: education_expenditure(%GDP)

In [98]:
#Mostarmos el valor máximo
my_dataframe.loc[my_dataframe.groupby('Region')['education_expenditure(%GDP)'].idxmax()][['Country', 'Region', 'education_expenditure(%GDP)']]

,Country,Region,education_expenditure(%GDP)
2,NAMIBIA,AFRICA,9.6
0,SOLOMON ISLANDS,AUSTRALIA AND OCEANIA,12.8
4,BELIZE,CENTRAL AMERICA AND THE CARIBBEAN,8.7
27,KYRGYZSTAN,CENTRAL ASIA,6.2
26,MACAU,EAST AND SOUTHEAST ASIA,6.3
10,SWEDEN,EUROPE,7.2
7,SAUDI ARABIA,MIDDLE EAST,7.8
30,UNITED STATES,NORTH AMERICA,6.1
1,BOLIVIA,SOUTH AMERICA,9.8
13,BHUTAN,SOUTH ASIA,7.0


In [99]:
#Mostramos el valor mínimo
my_dataframe.loc[my_dataframe.groupby('Region')['education_expenditure(%GDP)'].idxmin()][['Country', 'Region', 'education_expenditure(%GDP)']]


,Country,Region,education_expenditure(%GDP)
175,SOMALIA,AFRICA,0.3
160,VANUATU,AUSTRALIA AND OCEANIA,2.2
171,HAITI,CENTRAL AMERICA AND THE CARIBBEAN,1.4
149,ARMENIA,CENTRAL ASIA,2.8
172,PAPUA NEW GUINEA,EAST AND SOUTHEAST ASIA,1.4
137,ALBANIA,EUROPE,3.1
169,LEBANON,MIDDLE EAST,1.7
94,MEXICO,NORTH AMERICA,4.3
173,VENEZUELA,SOUTH AMERICA,1.3
167,SRI LANKA,SOUTH ASIA,1.9


In [100]:
#Mostramos la media
my_dataframe.groupby('Region', as_index=False)['education_expenditure(%GDP)'].mean()

,Region,education_expenditure(%GDP)
0,AFRICA,4.268000
1,AUSTRALIA AND OCEANIA,6.314286
2,CENTRAL AMERICA AND THE CARIBBEAN,4.555000
3,CENTRAL ASIA,4.333333
4,EAST AND SOUTHEAST ASIA,3.633333
5,EUROPE,5.151429
6,MIDDLE EAST,4.528571
7,NORTH AMERICA,5.200000
8,SOUTH AMERICA,4.808333
9,SOUTH ASIA,3.850000


Segunda varaible: power_generation(kW)

In [101]:
#Mostarmos el valor máximo
my_dataframe.loc[my_dataframe.groupby('Region')['power_generation(kW)'].idxmax()][['Country', 'Region', 'power_generation(kW)']]


,Country,Region,power_generation(kW)
17,SOUTH AFRICA,AFRICA,65989000
28,AUSTRALIA,AUSTRALIA AND OCEANIA,108193000
81,DOMINICAN REPUBLIC,CENTRAL AMERICA AND THE CARIBBEAN,6581000
117,RUSSIA,CENTRAL ASIA,301926000
118,CHINA,EAST AND SOUTHEAST ASIA,2949000000
78,GERMANY,EUROPE,275658000
7,SAUDI ARABIA,MIDDLE EAST,119620000
30,UNITED STATES,NORTH AMERICA,1235000000
31,BRAZIL,SOUTH AMERICA,240251000
87,INDIA,SOUTH ASIA,499136000


In [102]:
#Mostramos el valor mínimo
my_dataframe.loc[my_dataframe.groupby('Region')['power_generation(kW)'].idxmin()][['Country', 'Region', 'power_generation(kW)']]


,Country,Region,power_generation(kW)
62,SAO TOME AND PRINCIPE,AFRICA,29000
18,TONGA,AUSTRALIA AND OCEANIA,34000
47,DOMINICA,CENTRAL AMERICA AND THE CARIBBEAN,41000
27,KYRGYZSTAN,CENTRAL ASIA,3944000
96,TIMOR_LESTE,EAST AND SOUTHEAST ASIA,277000
23,MOLDOVA,EUROPE,779000
54,WEST BANK,MIDDLE EAST,352000
94,MEXICO,NORTH AMERICA,105586000
86,GUYANA,SOUTH AMERICA,259000
40,MALDIVES,SOUTH ASIA,432000


In [103]:
#Mostramos la media
my_dataframe.groupby('Region', as_index=False)['power_generation(kW)'].mean()

,Region,power_generation(kW)
0,AFRICA,4.619820e+06
1,AUSTRALIA AND OCEANIA,1.706100e+07
2,CENTRAL AMERICA AND THE CARIBBEAN,1.580500e+06
3,CENTRAL ASIA,4.239578e+07
4,EAST AND SOUTHEAST ASIA,2.108446e+08
5,EUROPE,4.027509e+07
6,MIDDLE EAST,3.381729e+07
7,NORTH AMERICA,5.008580e+08
8,SOUTH AMERICA,3.550408e+07
9,SOUTH ASIA,7.211612e+07


Tercera variable: Btu/person_consumption($)

In [104]:
#Mostarmos el valor máximo
my_dataframe.loc[my_dataframe.groupby('Region')['Btu/person_consumption($)'].idxmax()][['Country', 'Region', 'Btu/person_consumption($)']]


,Country,Region,Btu/person_consumption($)
56,SEYCHELLES,AFRICA,98847000
28,AUSTRALIA,AUSTRALIA AND OCEANIA,223158000
44,ARUBA,CENTRAL AMERICA AND THE CARIBBEAN,153952000
136,TURKMENISTAN,CENTRAL ASIA,261142000
147,SINGAPORE,EAST AND SOUTHEAST ASIA,643259000
35,MALTA,EUROPE,234698000
132,QATAR,MIDDLE EAST,814308000
59,CANADA,NORTH AMERICA,311599000
68,ARGENTINA,SOUTH AMERICA,78496000
13,BHUTAN,SOUTH ASIA,64082000


In [105]:
#Mostramos el valor mínimo
my_dataframe.loc[my_dataframe.groupby('Region')['Btu/person_consumption($)'].idxmin()][['Country', 'Region', 'Btu/person_consumption($)']]

,Country,Region,Btu/person_consumption($)
175,SOMALIA,AFRICA,649000
0,SOLOMON ISLANDS,AUSTRALIA AND OCEANIA,5655000
171,HAITI,CENTRAL AMERICA AND THE CARIBBEAN,3486000
37,TAJIKISTAN,CENTRAL ASIA,16192000
96,TIMOR_LESTE,EAST AND SOUTHEAST ASIA,6825000
137,ALBANIA,EUROPE,27407000
54,WEST BANK,MIDDLE EAST,14991000
94,MEXICO,NORTH AMERICA,57539000
130,PARAGUAY,SOUTH AMERICA,25733000
145,AFGHANISTAN,SOUTH ASIA,3380000


In [106]:
#Mostramos la media
my_dataframe.groupby('Region', as_index=False)['Btu/person_consumption($)'].mean()

,Region,Btu/person_consumption($)
0,AFRICA,1.412284e+07
1,AUSTRALIA AND OCEANIA,6.221671e+07
2,CENTRAL AMERICA AND THE CARIBBEAN,5.587300e+07
3,CENTRAL ASIA,1.039161e+08
4,EAST AND SOUTHEAST ASIA,1.186874e+08
5,EUROPE,1.131422e+08
6,MIDDLE EAST,2.406509e+08
7,NORTH AMERICA,2.158707e+08
8,SOUTH AMERICA,4.641467e+07
9,SOUTH ASIA,2.350638e+07


Cuarta variable: financial_assets_value($)

In [107]:
#Mostarmos el valor máximo
my_dataframe.loc[my_dataframe.groupby('Region')['financial_assets_value($)'].idxmax()][['Country', 'Region', 'financial_assets_value($)']]

,Country,Region,financial_assets_value($)
12,ALGERIA,AFRICA,81217000000
28,AUSTRALIA,AUSTRALIA AND OCEANIA,61703000000
141,GUATEMALA,CENTRAL AMERICA AND THE CARIBBEAN,21311000000
117,RUSSIA,CENTRAL ASIA,597217000000
118,CHINA,EAST AND SOUTHEAST ASIA,3450000000000
57,SWITZERLAND,EUROPE,863892000000
7,SAUDI ARABIA,MIDDLE EAST,457949000000
30,UNITED STATES,NORTH AMERICA,773426000000
31,BRAZIL,SOUTH AMERICA,355021000000
87,INDIA,SOUTH ASIA,627793000000


In [108]:
#Mostramos el valor mínimo
my_dataframe.loc[my_dataframe.groupby('Region')['financial_assets_value($)'].idxmin()][['Country', 'Region', 'financial_assets_value($)']]

,Country,Region,financial_assets_value($)
175,SOMALIA,AFRICA,30450000
18,TONGA,AUSTRALIA AND OCEANIA,396530000
47,DOMINICA,CENTRAL AMERICA AND THE CARIBBEAN,183530000
27,KYRGYZSTAN,CENTRAL ASIA,3237000000
96,TIMOR_LESTE,EAST AND SOUTHEAST ASIA,781995000
35,MALTA,EUROPE,1223000000
54,WEST BANK,MIDDLE EAST,1323000000
59,CANADA,NORTH AMERICA,117551000000
86,GUYANA,SOUTH AMERICA,895275000
40,MALDIVES,SOUTH ASIA,590523000


In [109]:
#Mostramos la media
my_dataframe.groupby('Region', as_index=False)['financial_assets_value($)'].mean()

,Region,financial_assets_value($)
0,AFRICA,7.087900e+09
1,AUSTRALIA AND OCEANIA,1.155909e+10
2,CENTRAL AMERICA AND THE CARIBBEAN,4.447386e+09
3,CENTRAL ASIA,8.023244e+10
4,EAST AND SOUTHEAST ASIA,3.723743e+11
5,EUROPE,8.739651e+10
6,MIDDLE EAST,1.002204e+11
7,NORTH AMERICA,3.684313e+11
8,SOUTH AMERICA,4.994452e+10
9,SOUTH ASIA,8.624625e+10


# CREATING 5 CATEGORIES FOR EACH VARIABLE

In [110]:
New_dataframe=my_dataframe.copy()

In [111]:
cuantil = list(my_dataframe['education_expenditure(%GDP)'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
cuantil

[0.3, 3.1, 3.9, 4.9, 5.9, 12.8]

In [112]:
New_dataframe['education_categorie']=pd.cut(New_dataframe['education_expenditure(%GDP)'],bins=cuantil,labels=['very_low','low','medium','high', 'very_high'],ordered=True, include_lowest=True)
New_dataframe.head()

,Country,Region,education_expenditure(%GDP),power_generation(kW),Btu/person_consumption($),financial_assets_value($),education_categorie
0,SOLOMON ISLANDS,AUSTRALIA AND OCEANIA,12.8,37000,5655000,688220000,very_high
1,BOLIVIA,SOUTH AMERICA,9.8,4375000,29340000,1800000000,very_high
2,NAMIBIA,AFRICA,9.6,646000,21734000,2956000000,very_high
3,SIERRA LEONE,AFRICA,9.1,149000,2301000,495699000,very_high
4,BELIZE,CENTRAL AMERICA AND THE CARIBBEAN,8.7,220000,30752000,473729000,very_high


In [113]:
cuantil2 = list(my_dataframe['power_generation(kW)'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
cuantil2

[29000.0, 352000.0, 2344000.0, 7419000.0, 31339000.0, 2949000000.0]

In [114]:
New_dataframe['generation_categorie']=pd.cut(New_dataframe['power_generation(kW)'],bins=cuantil2,labels=['very_low','low','medium','high', 'very_high'],ordered=True, include_lowest=True)
New_dataframe.head()

,Country,Region,education_expenditure(%GDP),power_generation(kW),Btu/person_consumption($),financial_assets_value($),education_categorie,generation_categorie
0,SOLOMON ISLANDS,AUSTRALIA AND OCEANIA,12.8,37000,5655000,688220000,very_high,very_low
1,BOLIVIA,SOUTH AMERICA,9.8,4375000,29340000,1800000000,very_high,medium
2,NAMIBIA,AFRICA,9.6,646000,21734000,2956000000,very_high,low
3,SIERRA LEONE,AFRICA,9.1,149000,2301000,495699000,very_high,very_low
4,BELIZE,CENTRAL AMERICA AND THE CARIBBEAN,8.7,220000,30752000,473729000,very_high,very_low


In [115]:
cuantil3 = list(my_dataframe['Btu/person_consumption($)'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
cuantil3

[649000.0, 8303000.0, 29305000.0, 64082000.0, 113163000.0, 814308000.0]

In [116]:
New_dataframe['consumption_categorie']=pd.cut(New_dataframe['Btu/person_consumption($)'],bins=cuantil3,labels=['very_low','low','medium','high', 'very_high'],ordered=True, include_lowest=True)
New_dataframe.head()

,Country,Region,education_expenditure(%GDP),power_generation(kW),Btu/person_consumption($),financial_assets_value($),education_categorie,generation_categorie,consumption_categorie
0,SOLOMON ISLANDS,AUSTRALIA AND OCEANIA,12.8,37000,5655000,688220000,very_high,very_low,very_low
1,BOLIVIA,SOUTH AMERICA,9.8,4375000,29340000,1800000000,very_high,medium,medium
2,NAMIBIA,AFRICA,9.6,646000,21734000,2956000000,very_high,low,low
3,SIERRA LEONE,AFRICA,9.1,149000,2301000,495699000,very_high,very_low,very_low
4,BELIZE,CENTRAL AMERICA AND THE CARIBBEAN,8.7,220000,30752000,473729000,very_high,very_low,medium


In [117]:
cuantil4 = list(my_dataframe['financial_assets_value($)'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
cuantil4

[30450000.0,
 781995000.0,
 3624000000.0,
 13730000000.0,
 69830000000.0,
 3450000000000.0]

In [118]:
New_dataframe['financial_categorie']=pd.cut(New_dataframe['financial_assets_value($)'],bins=cuantil4,labels=['very_low','low','medium','high', 'very_high'],ordered=True, include_lowest=True)
New_dataframe.head()

,Country,Region,education_expenditure(%GDP),power_generation(kW),Btu/person_consumption($),financial_assets_value($),education_categorie,generation_categorie,consumption_categorie,financial_categorie
0,SOLOMON ISLANDS,AUSTRALIA AND OCEANIA,12.8,37000,5655000,688220000,very_high,very_low,very_low,very_low
1,BOLIVIA,SOUTH AMERICA,9.8,4375000,29340000,1800000000,very_high,medium,medium,low
2,NAMIBIA,AFRICA,9.6,646000,21734000,2956000000,very_high,low,low,low
3,SIERRA LEONE,AFRICA,9.1,149000,2301000,495699000,very_high,very_low,very_low,very_low
4,BELIZE,CENTRAL AMERICA AND THE CARIBBEAN,8.7,220000,30752000,473729000,very_high,very_low,medium,very_low


Reordenamos las categorías

In [119]:
cat1 = New_dataframe['education_categorie']
New_dataframe=New_dataframe.drop(columns=['education_categorie'])
New_dataframe.insert(3, 'education_categorie', cat1 )
New_dataframe

,Country,Region,education_expenditure(%GDP),education_categorie,power_generation(kW),Btu/person_consumption($),financial_assets_value($),generation_categorie,consumption_categorie,financial_categorie
0,SOLOMON ISLANDS,AUSTRALIA AND OCEANIA,12.8,very_high,37000,5655000,688220000,very_low,very_low,very_low
1,BOLIVIA,SOUTH AMERICA,9.8,very_high,4375000,29340000,1800000000,medium,medium,low
2,NAMIBIA,AFRICA,9.6,very_high,646000,21734000,2956000000,low,low,low
3,SIERRA LEONE,AFRICA,9.1,very_high,149000,2301000,495699000,very_low,very_low,very_low
4,BELIZE,CENTRAL AMERICA AND THE CARIBBEAN,8.7,very_high,220000,30752000,473729000,very_low,medium,very_low
...,...,...,...,...,...,...,...,...,...,...
171,HAITI,CENTRAL AMERICA AND THE CARIBBEAN,1.4,very_low,472000,3486000,2586000000,low,very_low,low
172,PAPUA NEW GUINEA,EAST AND SOUTHEAST ASIA,1.4,very_low,1148000,8781000,3901000000,low,low,medium
173,VENEZUELA,SOUTH AMERICA,1.3,very_low,33493000,54474000,9794000000,very_high,medium,medium
174,NIGERIA,AFRICA,0.5,very_low,4094000,7993000,35564000000,medium,very_low,high


In [120]:
cat2 = New_dataframe['generation_categorie']
New_dataframe=New_dataframe.drop(columns=['generation_categorie'])
New_dataframe.insert(5, 'generation_categorie', cat2 )
New_dataframe

,Country,Region,education_expenditure(%GDP),education_categorie,power_generation(kW),generation_categorie,Btu/person_consumption($),financial_assets_value($),consumption_categorie,financial_categorie
0,SOLOMON ISLANDS,AUSTRALIA AND OCEANIA,12.8,very_high,37000,very_low,5655000,688220000,very_low,very_low
1,BOLIVIA,SOUTH AMERICA,9.8,very_high,4375000,medium,29340000,1800000000,medium,low
2,NAMIBIA,AFRICA,9.6,very_high,646000,low,21734000,2956000000,low,low
3,SIERRA LEONE,AFRICA,9.1,very_high,149000,very_low,2301000,495699000,very_low,very_low
4,BELIZE,CENTRAL AMERICA AND THE CARIBBEAN,8.7,very_high,220000,very_low,30752000,473729000,medium,very_low
...,...,...,...,...,...,...,...,...,...,...
171,HAITI,CENTRAL AMERICA AND THE CARIBBEAN,1.4,very_low,472000,low,3486000,2586000000,very_low,low
172,PAPUA NEW GUINEA,EAST AND SOUTHEAST ASIA,1.4,very_low,1148000,low,8781000,3901000000,low,medium
173,VENEZUELA,SOUTH AMERICA,1.3,very_low,33493000,very_high,54474000,9794000000,medium,medium
174,NIGERIA,AFRICA,0.5,very_low,4094000,medium,7993000,35564000000,very_low,high


In [121]:
cat3 = New_dataframe['consumption_categorie']
New_dataframe=New_dataframe.drop(columns=['consumption_categorie'])
New_dataframe.insert(7, 'consumption_categorie', cat3 )
New_dataframe

,Country,Region,education_expenditure(%GDP),education_categorie,power_generation(kW),generation_categorie,Btu/person_consumption($),consumption_categorie,financial_assets_value($),financial_categorie
0,SOLOMON ISLANDS,AUSTRALIA AND OCEANIA,12.8,very_high,37000,very_low,5655000,very_low,688220000,very_low
1,BOLIVIA,SOUTH AMERICA,9.8,very_high,4375000,medium,29340000,medium,1800000000,low
2,NAMIBIA,AFRICA,9.6,very_high,646000,low,21734000,low,2956000000,low
3,SIERRA LEONE,AFRICA,9.1,very_high,149000,very_low,2301000,very_low,495699000,very_low
4,BELIZE,CENTRAL AMERICA AND THE CARIBBEAN,8.7,very_high,220000,very_low,30752000,medium,473729000,very_low
...,...,...,...,...,...,...,...,...,...,...
171,HAITI,CENTRAL AMERICA AND THE CARIBBEAN,1.4,very_low,472000,low,3486000,very_low,2586000000,low
172,PAPUA NEW GUINEA,EAST AND SOUTHEAST ASIA,1.4,very_low,1148000,low,8781000,low,3901000000,medium
173,VENEZUELA,SOUTH AMERICA,1.3,very_low,33493000,very_high,54474000,medium,9794000000,medium
174,NIGERIA,AFRICA,0.5,very_low,4094000,medium,7993000,very_low,35564000000,high


In [122]:
New_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Country                      176 non-null    object  
 1   Region                       176 non-null    object  
 2   education_expenditure(%GDP)  176 non-null    float64 
 3   education_categorie          176 non-null    category
 4   power_generation(kW)         176 non-null    int64   
 5   generation_categorie         176 non-null    category
 6   Btu/person_consumption($)    176 non-null    int64   
 7   consumption_categorie        176 non-null    category
 8   financial_assets_value($)    176 non-null    int64   
 9   financial_categorie          176 non-null    category
dtypes: category(4), float64(1), int64(3), object(2)
memory usage: 9.9+ KB


Para salvaguardar las columnas category, utilizamos pikle. Pero también en formato csv

In [123]:
New_dataframe.to_csv("countrydata.csv",index=False )
New_dataframe.to_pickle("countrydata.pkl")